In [28]:
import pyaudio
import wave

CHUNK = 1024*4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:404:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:404:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_a52.so (/home/fall3n/anaconda3/envs/Tech/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/libjxl.so.0.11))
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_a52.so (/home/fall3n/anaconda3/envs/Tech/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: 

* recording
* done recording


In [29]:
from scipy.io import wavfile
from scipy.io import wavfile
import noisereduce as nr
import numpy as np
import noisereduce as nr
import numpy as np

# load data
rate, data = wavfile.read("output.wav")

# if stereo, convert to mono by averaging channels
if data.ndim > 1:
    data = np.mean(data, axis=1)

# perform noise reduction
reduced_noise = nr.reduce_noise(y=data, sr=rate)

# save output
wavfile.write("mywav_reduced_noise.wav", rate, reduced_noise.astype(np.int16))



In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-large-v3.5"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    dtype=torch_dtype,
    device=device,
)

result = pipe("output.wav")
print(result["text"])

Device set to use cuda:0
`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.


Hello, I am Parishwarya and I am into machine learning.


In [ ]:
import gc
import torch

# Delete the model object
del model
del pipe

# Run garbage collection
gc.collect()

# Clear CUDA cache if running on GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()


In [ ]:
from transformers import pipeline
import nltk

# Grammar correction model
corrector = pipeline("text2text-generation", model="prithivida/grammar_error_correcter_v1")

def grammar_score(sentence):
    corrected = corrector(sentence, max_length=128)[0]['generated_text']
    # Simple token-based edit distance
    dist = nltk.edit_distance(sentence.split(), corrected.split())
    max_len = max(len(sentence.split()), len(corrected.split()))
    score = 1 - dist / max_len  # normalized accuracy score
    return corrected, round(score * 100, 2)

s = "He is going to school."
corrected, score = grammar_score(s)
print("Original:", s)
print("Corrected:", corrected)
print("Grammar Accuracy Score:", score, "%")


import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a tech interviewer and you only ask them question and do not have to say anything else and you have to ask the candidate question based on what they specialize in. You only have to ask them about tech interview question such as DSA or ML related question. If they give you information about any project that they have done then you will have to ask them question regarding that project."
    ,
    },
    {"role": "user", "content": "Hello I am Priyanshu and I am praticing Machine learning and Deep learning and I have done a project on CNN in which I used CNN to classify digits"},
]
    # {"role": "user", "content": result["text"]},
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)

In [2]:


# prepare the model input
prompt = "Hello I am Priyanshu and I am praticing Machine learning and Deep learning and I have done a project on CNN in which I used CNN to classify digits"
messages = [
    {
        "role": "system",
        "content": "You are a tech interviewer and you only ask them question and do not have to say anything else and you have to ask the candidate question based on what they specialize in. You only have to ask them about tech interview question such as DSA or ML related question. If they give you information about any project that they have done then you will have to ask them question regarding that project. Ask the canditate around 10 questions"
    ,
    },
    {"role": "user", "content":textIDK }
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True)
content = content + "\n"

print("thinking content:", thinking_content)
print("content:", content)
# prepare the model input
prompt = "Hello I am Priyanshu and I am praticing Machine learning and Deep learning and I have done a project on CNN in which I used CNN to classify digits"
messages = [
    {
        "role": "system",
        "content": "You are a tech interviewer and you only ask them question and do not have to say anything else and you have to ask the candidate question based on what they specialize in. You only have to ask them about tech interview question such as DSA or ML related question. If they give you information about any project that they have done then you will have to ask them question regarding that project. Ask the canditate around 10 questions"
    ,
    },
    {"role": "user", "content":textIDK }
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True)
content = content + "\n"

print("thinking content:", thinking_content)
print("content:", content)


NameError: name 'textIDK' is not defined